# はじめに

Cifarとは、一般物体認識のベンチマークです。CifarはCifar-10とCifar-100に分かれ、数字はクラス数を表します。10は10クラス、100は100クラスをそれぞれ表すわけです。とはいえ、あまり良いCPUを持っていない中で演習をするので、今回はCifar-10で、一般物体認識をしていきましょう。

![Cifar-10の一部](http://cdn-ak.f.st-hatena.com/images/fotolife/a/aidiary/20151014/20151014211729.png)

では、まずCifar-10のデータセットの読み込みから始めましょう。

```
$ wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
```

上記のコマンドで、cifar-10のpython versionをダウンロードして展開してみましょう。

```
$ tar xzf cifar-10-python.tar.gz
```

これで作られたcifar-10-batches-pyというディレクトリを使って作業していきます。

```
$ ls cifar-10-batches-py
batches.meta data_batch_1 data_batch_2 data_batch_3 data_batch_4 data_batch_5 readme.html  test_batch
```

ディレクトリの中身を見ると、batch_Xと呼ばれる5つのデータとtest_batch, readmeとなんだかわからないmetaファイルが入っています。readmeを読んでみましょう。どうやら、cifarのページそのままのようです。見るべき部分を抜粋すると、

> The archive contains the files data_batch_1, data_batch_2, ..., data_batch_5, as well as test_batch. Each of these files is a Python "pickled" object produced with cPickle. Here is a Python routine which will open such a file and return a dictionary:

```
def unpickle(file):
    import cPickle
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict
```

と書いてあるので、指示通りに作ってみましょう。

In [8]:
import os
import cPickle as pickle

for root, dirs, files in os.walk('cifar-10-batches-py'):
    for fname in files:
        if fname.find('data_batch_') >= 0:
            path = os.path.join(root, fname)
            with open(path, 'rb') as fp:
                data = pickle.load(fp)
                print fname, ":", type(data)
                for k, v in data.items():
                    print '\t', k, ':', type(v)

data_batch_1 : <type 'dict'>
	data : <type 'numpy.ndarray'>
	labels : <type 'list'>
	batch_label : <type 'str'>
	filenames : <type 'list'>
data_batch_2 : <type 'dict'>
	data : <type 'numpy.ndarray'>
	labels : <type 'list'>
	batch_label : <type 'str'>
	filenames : <type 'list'>
data_batch_3 : <type 'dict'>
	data : <type 'numpy.ndarray'>
	labels : <type 'list'>
	batch_label : <type 'str'>
	filenames : <type 'list'>
data_batch_4 : <type 'dict'>
	data : <type 'numpy.ndarray'>
	labels : <type 'list'>
	batch_label : <type 'str'>
	filenames : <type 'list'>
data_batch_5 : <type 'dict'>
	data : <type 'numpy.ndarray'>
	labels : <type 'list'>
	batch_label : <type 'str'>
	filenames : <type 'list'>


さてデータの素性が分かりました。各バッチには、辞書型のオブジェクトが入っていて、その要素にdata, labels, batch_labels, filenamesが入っているらしいですね。では、それぞれの要素を見ていきましょう。今回は、data_batch_1のみを見ていくことにして、また、labels, filenamesにはたくさんのデータが入っていそうなので、最初の10要素だけ表示します。

In [13]:
with open('cifar-10-batches-py/data_batch_1', 'rb') as fp:
    data = pickle.load(fp)
    print 'data: ', data['data'].shape
    print 'labels: ', data['labels'][:10]
    print 'batch_label: ',  data['batch_label']
    print 'filenames: ', data['filenames'][:10]


data:  (10000, 3072)
labels:  [6, 9, 9, 4, 1, 1, 2, 7, 8, 3]
batch_label:  training batch 1 of 5
filenames:  ['leptodactylus_pentadactylus_s_000004.png', 'camion_s_000148.png', 'tipper_truck_s_001250.png', 'american_elk_s_001521.png', 'station_wagon_s_000293.png', 'coupe_s_001735.png', 'cassowary_s_001300.png', 'cow_pony_s_001168.png', 'sea_boat_s_001584.png', 'tabby_s_001355.png']


ここでの出力から、dataには一つ3072次元のデータが10000個入っていて、labelはおそらく0-9の数字が入っているリスト型であり、batch_labelはこのバッチファイルの説明で、pngファイルは、それぞれのエントリのファイル名っぽいことがわかりました。ここで、batch_label, filenamesは学習に必要ないから省くことにします。

まあ、そんなこと調べんでも書いてありますが、

> data -- a 10000x3072 numpy array of uint8s. Each row of the array stores a 32x32 colour image. The first 1024 entries contain the red channel values, the next 1024 the green, and the final 1024 the blue. The image is stored in row-major order, so that the first 32 entries of the array are the red channel values of the first row of the image.
> labels -- a list of 10000 numbers in the range 0-9. The number at index i indicates the label of the ith image in the array data.

さて、本当は、バッチファイル群をぜんぶ結合させて一つの学習セットを作ったほうが良いのでしょうが、次元数がMNISTの4倍ちかくあるので、この時点で4倍の計算量が必要であることがわかります。これを、全部学習するのはサンプルの段階では得策ではありません。とりあえず、batch一つで試してみましょう。ということで、このバッチファイルの中にあるデータに偏りがないかを調べていきます。

In [14]:
for i in xrange(10):
    print '[%d]: ' % i, data['labels'].count(i)

[0]:  1005
[1]:  974
[2]:  1032
[3]:  1016
[4]:  999
[5]:  937
[6]:  1030
[7]:  1001
[8]:  1025
[9]:  981


どうやら、各要素1000個前後に分かれており、それほどのばらつきはなさそうです。ということで、data_batch_1とtest_batchを使って、MLPに与えるデータ・セットを作っていきましょう。

# Cifar10とMLP（「ハリーポッターと賢者の石」的な・・）

さて、ではこれらのデータをこれまたこれまで作ったMLPで学習しましょう。MLPの定義を思い出すと、

```
class MLP(object):
	def __init__(
		self,
		data,
		target,
		n_inputs=784,
		n_hidden=784,
		n_outputs=10,
		gpu=-1
	):
...
```

ということで、入力と出力の次元数を調整できるので、次元数が増えても普通に対応できそうです。なので、後は、これに当てはまるように、データを整形してあげれば良いんですね。まずは、main関数ででてくるdata, targetを作っていきましょう。

In [24]:
import numpy

with open('cifar-10-batches-py/data_batch_1', 'rb') as fp:
    train = pickle.load(fp)

data_train = train['data'].astype(numpy.float32)
data_train /= 255.
target_train = numpy.array(train['labels']).astype(numpy.int32)

with open('cifar-10-batches-py/test_batch', 'rb') as fp:
    test = pickle.load(fp)

data_test = test['data'].astype(numpy.float32)
data_test /= 255.
target_test = numpy.array(test['labels']).astype(numpy.int32)

print 'data_train: ', data_train.shape, data_train[:10]
print 'target_train: ', target_train.shape, target_train[:10]
print 'data_test: ', data_test.shape, data_train[:10]
print 'target_test: ', target_test.shape, target_train[:10]


data_train:  (10000, 3072) [[ 0.23137255  0.16862746  0.19607843 ...,  0.54901963  0.32941177
   0.28235295]
 [ 0.60392159  0.49411765  0.41176471 ...,  0.54509807  0.55686277
   0.56470591]
 [ 1.          0.99215686  0.99215686 ...,  0.32549021  0.32549021
   0.32941177]
 ..., 
 [ 0.10980392  0.11764706  0.12941177 ...,  0.39215687  0.3882353
   0.3764706 ]
 [ 0.52549022  0.51372552  0.50196081 ...,  0.53333336  0.53725493
   0.5411765 ]
 [ 0.49019608  0.43137255  0.40000001 ...,  0.32156864  0.32941177
   0.33725491]]
target_train:  (10000,) [6 9 9 4 1 1 2 7 8 3]
data_test:  (10000, 3072) [[ 0.23137255  0.16862746  0.19607843 ...,  0.54901963  0.32941177
   0.28235295]
 [ 0.60392159  0.49411765  0.41176471 ...,  0.54509807  0.55686277
   0.56470591]
 [ 1.          0.99215686  0.99215686 ...,  0.32549021  0.32549021
   0.32941177]
 ..., 
 [ 0.10980392  0.11764706  0.12941177 ...,  0.39215687  0.3882353
   0.3764706 ]
 [ 0.52549022  0.51372552  0.50196081 ...,  0.53333336  0.53725493
 

ここでは、便利関数train_test_splitを使いませんでした。何故かと言うとlabelデータを見ると既にデータがシャッフルされているからです。厳密に行いたい場合は、再シャッフルをすると良いかもしれませんが、サンプルとして使う分には問題無いでしょう。では、いよいよ、MLPで学習してみましょう。ここからは、MLP.pyのmain関数をほぼほぼコピーしながら進めることができます。

In [31]:
import time
import logging
reload(logging)
logging.basicConfig(level=logging.DEBUG)

from mlp import MLP

data = data_train, data_test
target = target_train, target_test

start_time = time.time()

mlp = MLP(data=data, target=target, gpu=-1, n_inputs=3072, n_outputs=10)
mlp.train_and_test(n_epoch=1)

end_time = time.time()

print("time = {} min".format((end_time - start_time) / 60.0))

INFO:root:epoch 1
INFO:root:train mean loss=2.26053388834, accuracy=0.178700000271
INFO:root:test mean loss=2.01086740851, accuracy=0.276900000721


time = 0.184891434511 min


さて、ここまでが簡単なCifar-10におけるデータの処理方法でした。しかしながら、見てみると分かる通り、識別率は20%前後と一般物体認識はMNISTと比べると、難しい課題であることがわかります。ということで、画像認識に特化していると言われるCNNを使って、Cifar-10の識別をしてみましょう。

# Cifar10とCNN(「ハリーポッターと不死鳥の騎士団」的な・・)

CNNを使って、Cifar10を学習する際の注意点は以下の2つです。

* CNNModelの変更。画像サイズ・チャネル数が変わるので、MLPの展開をしたときの次元数が代わります
* データを2次元から4次元（データ数xチャネル数x縦x横）似直す必要があります

ということで、これらを変更して、Cifar10を学習できるCNNのクラスを作りましょう。というのが本日の課題です。頑張って取り組んでいきましょう！